<a href="https://colab.research.google.com/github/Dhormir/PotatoMachine-Taller1/blob/main/challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="30%" /></center>

<hr style="height:2px;border:none"/>
<h1 align='center'> INF-393 Aprendizaje Automático </h1>

<H3 align='center'> Tarea/Taller 1 </H3>
<hr style="height:2px;border:none"/>

**Integrantes**


*   Domingo Benoit
*   Tomás Berríos



**Temas**  

* Clasificadores Discriminativos Clásicos
* Clasificadores Generativos Clásicos
* Regresión Lineal y Regularización 
* Selección de Características (parcial, seguiremos en T2)
* Reducción de Dimensionalidad


<hr style="height:2px;border:none"/>


**Nota:** Al momento de realizar la experimentación, se buscó en la literatura que técnicas y modelos se ocupaban, y con eso se usaron durante la competencia modelos fuera de los estudiados, sin darnos cuenta que no estaba permitido, por lo que finalmente se optó por utilizar un modelo de regresión lineal, ajustando sus hiperparametros con una representación de texto. Se explica esto dado que durante el notebook, la mayor parte de código es al entrenamiento del modelo SVM.

#Imports

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, accuracy_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree

import pandas as pd
import numpy as np

import nltk

from nltk.corpus import stopwords

from nltk.stem.snowball import SnowballStemmer

from nltk.stem import WordNetLemmatizer 

#Dataset

In [9]:
print("Mounting your Google Drive ...")

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive',force_remount=True)
 
path = "/content/drive/My Drive/Defensa 1/Parte 2/dataset/"


Mounting your Google Drive ...
Mounted at /content/drive


In [10]:
train_name = 'train.csv'
data = pd.read_csv(path + train_name)

submit_name = "x_test.csv"
submit_csv = pd.read_csv(path + submit_name)

#Analisis exploratorio

In [11]:
data.head()

,title,category
0,Bikini Malla Lunares Colores Colección Verano...,WOMEN_SWIMWEAR
1,Kit 4 Bujias Ngk Ford F-100 3.0 Bp5efs,SPARK_PLUGS
2,Campera Rompeviento adidas Originals Hard Shel...,JACKETS_AND_COATS
3,Vulk - Danner Sblk/sg91 Polarized,SUNGLASSES
4,Calza Frizada Termica,PANTS


In [12]:
freq_class = data.groupby('category')['category'].count()
freq_class

category
ACTION_FIGURES                  2763
AUTOMOTIVE_CLUTCH_KITS          2036
AUTOMOTIVE_SIDE_VIEW_MIRRORS    2075
BABY_CAR_SEATS                  1732
BABY_STROLLERS                  2015
BACKPACKS                       2478
BOARD_GAMES                     2119
BOOKS                           2738
CAR_STEREOS                     2182
CATS_AND_DOGS_FOODS             2060
CEILING_LIGHTS                  2365
COMPUTER_MONITORS               1814
COMPUTER_PROCESSORS             1780
COSTUMES                        1921
DIECAST_VEHICLES                2149
DOLLS                           2404
ELECTRIC_GUITARS                2392
EYESHADOWS                      1789
FOOTBALL_SHIRTS                 2613
FOOTBALL_SHOES                  2028
HANDBAGS                        2570
HOME_HEATERS                    2073
INSTRUMENT_AMPLIFIERS           1720
JACKETS_AND_COATS               2639
LIGHT_BULBS                     2310
LIPSTICKS                       1682
MAGAZINES                    

In [13]:
print("Samples categoría menor representada:",np.min(np.array(freq_class)),
      "\nSamples categoría más representada:",np.max(np.array(freq_class)))

Samples categoría menor representada: 1677 
Samples categoría más representada: 2763


#Pre-procesamiento del texto

In [14]:
values = np.array(data['category'])

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

data.category = integer_encoded

titles = np.array(data.title)
categories = np.array(data.category)

In [15]:
nltk.download('wordnet')
nltk.download('stopwords')

tokenizer = TfidfVectorizer().build_tokenizer()#Return a function that splits a string into a sequence of tokens considering unicode characters
stemmer = SnowballStemmer("spanish") 
lemmatizer = WordNetLemmatizer()

remove_stops_here = True

def my_pre_processer(text):
    results = []
    for token in tokenizer(text):
        clean_token = token.lower().strip().strip('-').strip('_')
        if remove_stops_here and (clean_token in stopwords.words('spanish')):
          continue
        token_pro = stemmer.stem(clean_token) #podemos probar stemming en vez de lematizacion
        #token_pro = lemmatizer.lemmatize(clean_token)
        #token_pro = clean_token
        if not token_pro[0].isdigit(): 
            results.append(token_pro)
    return results

print(titles[0])
my_pre_processer(titles[0])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Bikini Malla Lunares  Colores Colección Verano 2014


['bikini', 'mall', 'lunar', 'color', 'coleccion', 'veran']

##Representaciones del texto

In [16]:
ngram_max = 1
max_features = 10000
max_df=0.8
min_df=1

vectorizer_Tfidf = TfidfVectorizer(stop_words='english', tokenizer=my_pre_processer, min_df=min_df, max_df=max_df,
                            binary=False, use_idf=True, smooth_idf=True, 
                            norm=None)

vectorizer_Tf = CountVectorizer(stop_words='english', tokenizer=my_pre_processer, min_df=min_df, max_df=max_df)

vectorizer_binary = CountVectorizer(stop_words='english', tokenizer=my_pre_processer, min_df=min_df, max_df=max_df,
                                binary=True)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(titles, categories, test_size=0.1, random_state=1337)

titles_tf = vectorizer_Tf.fit_transform(titles)
X_train_tf = vectorizer_Tf.transform(X_train)
X_test_tf = vectorizer_Tf.transform(X_test)
X_pred_tf = vectorizer_Tf.transform(np.array(submit_csv.title))

titles_binary = vectorizer_binary.fit_transform(titles)
X_train_binary = vectorizer_binary.transform(X_train)
X_test_binary = vectorizer_binary.transform(X_test)
X_pred_binary = vectorizer_binary.transform(np.array(submit_csv.title))

X_train_tfidf = vectorizer_Tfidf.fit_transform(X_train)
X_test_tfidf = vectorizer_Tfidf.transform(X_test)
X_pred_tfidf = vectorizer_Tfidf.transform(np.array(submit_csv.title))

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'alon', 'anoth', 'anyon', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'betw', 'describ', 'don', 'eith', 'elev', 'elsewher', 'everyon', 'everywher', 'fift', 'fir', 'fiv', 'form', 'furth', 'giv', 'hav', 'henc', 'hereaft', 'howev', 'hundr', 'inde', 'latt', 'mad', 'meanwhil', 'min', 'mor', 'moreov', 'mov', 'nam', 'neith', 'nev', 'nin', 'non', 'noon', 'nowher', 'otherwis', 'ourselv', 'ple', 'rath', 'sam', 'sid', 'sinc', 'sincer', 'som', 'someon', 'sometim', 'somewher', 'tak', 'themselv', 'thenc', 'ther', 'thereaft', 'therefor', 'thes', 'thos', 'thre', 'togeth', 'twelv', 'wer', 'whatev', 'whenc', 'whenev', 'wher', 'whereaft', 'wherev', 'wheth', 'whil', 'whith', 'whoev', 'whol', 'whos', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


##Representación elegida

In [ ]:
ngram_max = 1
max_features = 10000
max_df=0.8
min_df=1

vectorizer_Tfidf = TfidfVectorizer(stop_words='english', tokenizer=my_pre_processer, min_df=min_df, max_df=max_df,
                            binary=False, use_idf=True, smooth_idf=True, 
                            norm=None)

X_train, X_test, y_train, y_test = train_test_split(titles, categories, test_size=0.1, random_state=1337)

X_titles_tfidf = vectorizer_Tfidf.fit_transform(titles)

X_train_tfidf = vectorizer_Tfidf.transform(X_train)
X_test_tfidf = vectorizer_Tfidf.transform(X_test)
X_pred_tfidf = vectorizer_Tfidf.transform(np.array(submit_csv.title))

#Modelos

In [19]:
dt = DecisionTreeClassifier()
svm = LinearSVC(penalty='l2', max_iter=50000)
svm_balanced = LinearSVC(penalty='l2', class_weight='balanced', max_iter=50000)
nb = MultinomialNB()
lr = LogisticRegression(penalty='l2', class_weight='balanced', max_iter=50000)

#Training

##Elección del modelo

###Entrenamiento con GridSearchCV

Entrenamiento de GridSearchCV con representación binaria.

In [ ]:
C = [1000, 0.1, 0.01, 0.001, 0.0001]
penalty = ['l2']
max_iter = [50000]
class_weight = ['balanced']

model = LinearSVC()

grid = dict(C = C,
            penalty = penalty,
            max_iter = max_iter,
            class_weight = class_weight)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy', error_score=0)
grid_result = grid_search.fit(X_train_binary, y_train)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.975571 using {'C': 0.1, 'class_weight': 'balanced', 'max_iter': 50000, 'penalty': 'l2'}
0.952753 (0.003164) with: {'C': 1000, 'class_weight': 'balanced', 'max_iter': 50000, 'penalty': 'l2'}
0.975571 (0.001370) with: {'C': 0.1, 'class_weight': 'balanced', 'max_iter': 50000, 'penalty': 'l2'}
0.967980 (0.001403) with: {'C': 0.01, 'class_weight': 'balanced', 'max_iter': 50000, 'penalty': 'l2'}
0.948136 (0.002180) with: {'C': 0.001, 'class_weight': 'balanced', 'max_iter': 50000, 'penalty': 'l2'}
0.918718 (0.002183) with: {'C': 0.0001, 'class_weight': 'balanced', 'max_iter': 50000, 'penalty': 'l2'}


Entrenamiento de GridSearchCV con representación Tf.

In [ ]:
C = [1000, 0.1, 0.01, 0.001, 0.0001]
penalty = ['l2']
max_iter = [50000]
class_weight = ['balanced']

model = LinearSVC()

grid = dict(C = C,
            penalty = penalty,
            max_iter = max_iter,
            class_weight = class_weight)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy', error_score=0)
grid_result = grid_search.fit(X_train_tf, y_train)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.975526 using {'C': 0.1, 'class_weight': 'balanced', 'max_iter': 50000, 'penalty': 'l2'}
0.951979 (0.003576) with: {'C': 1000, 'class_weight': 'balanced', 'max_iter': 50000, 'penalty': 'l2'}
0.975526 (0.001212) with: {'C': 0.1, 'class_weight': 'balanced', 'max_iter': 50000, 'penalty': 'l2'}
0.967921 (0.001391) with: {'C': 0.01, 'class_weight': 'balanced', 'max_iter': 50000, 'penalty': 'l2'}
0.948206 (0.002069) with: {'C': 0.001, 'class_weight': 'balanced', 'max_iter': 50000, 'penalty': 'l2'}
0.918608 (0.002111) with: {'C': 0.0001, 'class_weight': 'balanced', 'max_iter': 50000, 'penalty': 'l2'}


Entrenamiento de GridSearchCV con representación Tf-idf al modelo LR.

In [ ]:
C = [1000, 0.1, 0.01, 0.001, 0.0001]
penalty = ['l2', 'l1']
max_iter = [50000]
class_weight = ['balanced']

model = LogisticRegression()

grid = dict(C = C,
            penalty = penalty,
            max_iter = max_iter,
            class_weight = class_weight)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy', error_score=0)
grid_result = grid_search.fit(X_train_tfidf, y_train)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

###Entrenamiento CV con diferentes modelos y representaciones de texto.

A continuación se utilizara un modelo de regresión logistica

In [20]:
#Probando LR con diferentes representaciones
cv_scores_bn = cross_val_score(lr, X_train_binary, y_train, cv = 10)
cv_scores_tf = cross_val_score(lr, X_train_tf, y_train, cv = 10)
cv_scores_tfidf = cross_val_score(lr, X_train_tfidf, y_train, cv = 10)
print("Modelo LR con diferentes representaciones:")
print("\nBinary:", np.mean(cv_scores_bn),
      "\nTf:", np.mean(cv_scores_tf),
      "\nTf-idf", np.mean(cv_scores_tfidf))

Modelo LR con diferentes representaciones:

Binary: 0.9719190159306781 
Tf: 0.9716806019915969 
Tf-idf 0.9720744960883705


En adelante se comparará modelos no estudiados aún junto con modelos de NB.

In [ ]:
cv_scores_dt = cross_val_score(dt, X_train_binary, y_train, cv = 10)
cv_scores_svm = cross_val_score(svm, X_train_binary, y_train, cv = 10)
cv_scores_svm_balanced = cross_val_score(svm_balanced, X_train_binary, y_train, cv = 10)
cv_score_nb = cross_val_score(nb, X_train_binary, y_train, cv = 10)
print("Representación Binary")
print("\nCV Decision Tree:", np.mean(cv_scores_dt),
      "\nCV Support Vector Machines:", np.mean(cv_scores_svm),
      "\nCV SVM con Penalizacion de Desbalanceo:", np.mean(cv_scores_svm_balanced),
      "\nCV NB:", np.mean(cv_score_nb))

Representación Binary

CV Decision Tree: 0.9447501874202464 
CV Support Vector Machines: 0.9743446114680735 
CV SVM con Penalizacion de Desbalanceo: 0.9744586344047702 
CV NB: 0.9653885484725228


In [ ]:
cv_scores_dt = cross_val_score(dt, X_train_tf, y_train, cv = 10)
cv_scores_svm = cross_val_score(svm, X_train_tf, y_train, cv = 10)
cv_scores_svm_balanced = cross_val_score(svm_balanced, X_train_tf, y_train, cv = 10)
cv_score_nb = cross_val_score(nb, X_train_tf, y_train, cv = 10)
print("Representación TF")
print("\nCV Decision Tree:", np.mean(cv_scores_dt),
      "\nCV Support Vector Machines:", np.mean(cv_scores_svm),
      "\nCV SVM con Penalizacion de Desbalanceo:", np.mean(cv_scores_svm_balanced),
      "\nCV NB:", np.mean(cv_score_nb))

Representación TF

CV Decision Tree: 0.9447087430921822 
CV Support Vector Machines: 0.9743446060960187 
CV SVM con Penalizacion de Desbalanceo: 0.9744897278577225 
CV NB: 0.9652434277852299


In [ ]:
cv_scores_dt = cross_val_score(dt, X_train_tfidf, y_train, cv = 10)
cv_scores_svm = cross_val_score(svm, X_train_tfidf, y_train, cv = 10)
cv_scores_svm_balanced = cross_val_score(svm_balanced, X_train_tfidf, y_train, cv = 10)
cv_score_nb = cross_val_score(nb, X_train_tfidf, y_train, cv = 10)
print("Representación TF-IDF")
print("\nCV Decision Tree:", np.mean(cv_scores_dt),
      "\nCV Support Vector Machines:", np.mean(cv_scores_svm),
      "\nCV SVM con Penalizacion de Desbalanceo:", np.mean(cv_scores_svm_balanced),
      "\nCV NB:", np.mean(cv_score_nb))

Representación TF-IDF

CV Decision Tree: 0.9448953177772381 
CV Support Vector Machines: 0.9628281949414669 
CV SVM con Penalizacion de Desbalanceo: 0.962651976503578 
CV NB: 0.9593660098663588


In [ ]:
df = pd.DataFrame(data = y_train, columns=["y"])

n_total = np.sum(df.groupby('y')['y'].count())
prior = np.array(df.groupby('y')['y'].count())/n_total

nb = MultinomialNB(class_prior=prior)
cv_score_nb = cross_val_score(nb, X_train_binary, y_train, cv = 10)
print(np.mean(cv_score_nb))

0.9653885484725228


##Entrenamiento del modelo final

###Entrenamiento con el Train set

In [ ]:
model = LogisticRegression(penalty='l2', class_weight='balanced', max_iter=50000, C=0.1)
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       277
           1       1.00      1.00      1.00       214
           2       0.99      0.99      0.99       197
           3       0.98      0.99      0.99       177
           4       0.99      0.98      0.99       176
           5       0.99      0.99      0.99       247
           6       0.93      0.92      0.92       210
           7       0.83      0.92      0.87       272
           8       1.00      0.98      0.99       200
           9       0.99      0.98      0.98       214
          10       0.91      0.92      0.92       239
          11       0.98      0.96      0.97       182
          12       0.99      0.99      0.99       159
          13       0.99      0.95      0.97       184
          14       0.95      0.99      0.97       225
          15       0.97      0.97      0.97       215
          16       0.97      0.99      0.98       261
          17       0.99    

###Entrenamiento con todos los datos

In [ ]:
model = LogisticRegression(penalty='l2', class_weight='balanced', max_iter=50000)
model.fit(X_titles_tfidf, categories)

LinearSVC(C=0.1, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=50000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

#Plotting

#Submit

In [ ]:
submit_csv

,Id,title,Predicted
0,0,Collar Marga Black,NaN
1,1,Microfono Condenser Rode Nt5,NaN
2,2,Apple Watch + Nike 42mm Silver Aluminum Case P...,NaN
3,3,Casco Abierto Shiro Sh-62 Monocolor Gris - Um,NaN
4,4,Jeans Negro Roto Chupin Hombre Ybr Jeans,NaN
...,...,...,...
71456,71456,Hot Wheels 15 Ford F150 Pickup 2015 # 119 So...,NaN
71457,71457,Memoria So-dimm Ddr3 2gb 1600mhz Pc3-12800s,NaN
71458,71458,Microprocesador Intel I7 2.93 Mhz Socket 1366,NaN
71459,71459,Jv Casco Ls2 Convertible Rebatible Ff 399 Val...,NaN


In [ ]:
predicted_class = model.predict(X_pred_tfidf)
inverted_predicted = label_encoder.inverse_transform(predicted_class)
submit_csv.Predicted = inverted_predicted
del submit_csv['title']
submit_csv

,Id,Predicted
0,0,NECKLACES
1,1,MICROPHONES
2,2,SMARTWATCHES
3,3,MOTORCYCLE_HELMETS
4,4,PANTS
...,...,...
71456,71456,DIECAST_VEHICLES
71457,71457,RAM_MEMORY_MODULES
71458,71458,COMPUTER_PROCESSORS
71459,71459,MOTORCYCLE_HELMETS


In [ ]:
submit_csv.to_csv('submission.csv', index = False, header=True)